In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
# Design variables
center_x =0
center_y =0
x = 0.6875+0.650 # qubit x spacing
y = 2.7+0.650 # qubit y spacing

In [3]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [30]:

design.delete_all_components()

# Transmons

In [5]:
qubits[3][0].options

{'pos_x': '1.3375mm',
 'pos_y': '1.675',
 'orientation': '0.0',
 'chip': 'Q_chip',
 'layer': '1',
 'connection_pads': {'c': {'pad_gap': '15um',
   'pad_width': '120um',
   'pad_height': '30um',
   'pad_cpw_shift': '0um',
   'pad_cpw_extent': '25um',
   'cpw_width': '10um',
   'cpw_gap': '6um',
   'cpw_extend': '100um',
   'pocket_extent': '5um',
   'pocket_rise': '0um',
   'loc_W': 0,
   'loc_H': -1},
  'b': {'pad_gap': '15um',
   'pad_width': '120um',
   'pad_height': '30um',
   'pad_cpw_shift': '0um',
   'pad_cpw_extent': '25um',
   'cpw_width': '10um',
   'cpw_gap': '6um',
   'cpw_extend': '100um',
   'pocket_extent': '5um',
   'pocket_rise': '0mm',
   'loc_W': 1,
   'loc_H': 1}},
 'pad_gap': '30um',
 'inductor_width': '20um',
 'pad_width': '455um',
 'pad_height': '90um',
 'pocket_width': '650um',
 'pocket_height': '650um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'hfss_inductance': '10nH',
 'hfss_capacitance': 0,
 'hfss_resistance': 0,
 'hfss_mesh_kw_jj': 7e-06,
 'q3d_

In [4]:
cp0 = dict(c = dict(loc_W=0, loc_H=-1, pad_width='120um'),
           b = dict(loc_W = +1, loc_H = +1, pad_width='120um'))
cp1 = dict(a = dict(loc_W=-1, loc_H=1, pad_width='120um'),
           c  = dict(loc_W=0, loc_H=-1, pad_width='120um'))
cp2= dict( a  = dict(loc_W=-1, loc_H=+1, pad_width='120um'),
           b = dict(loc_W = 1, loc_H = 1, pad_width='120um'))
cp3= dict(c = dict(loc_W=0, loc_H=-1, pad_width='120um'),
          b = dict(loc_W = 1, loc_H = 1, pad_width='120um', pocket_rise='0mm'))
cp4 = dict(c = dict(loc_W=0, loc_H=-1, pad_width='120um'),
           a  = dict(loc_W=-1, loc_H=+1, pad_width='120um'))
cp5 = dict(c = dict(loc_W=1, loc_H=-1, pad_width='120um'),
           a  = dict(loc_W=-1, loc_H=+1, pad_width='120um'))
cp678= dict(c = dict(loc_W=1, loc_H=-1, pad_width='120um', pocket_rise='0mm'),
          b = dict(loc_W = -1, loc_H = -1, pad_width='120um'))
cp9= dict(a  = dict(loc_W=1, loc_H=+1, pad_width='120um'),
          b = dict(loc_W = -1, loc_H = -1, pad_width='120um', pocket_rise='0mm'))
cp=[cp0,cp1,cp2,cp3,cp4,cp5,cp678,cp9]                                          
                                                
space=[-2*x,-1*x,0,x,2*x] 
qubits=[]                       
for i,j in zip(range(0,5), space):
    a=[TransmonPocket6(design,'Q'+str(i), options=dict(pos_x=f"{center_x+j}mm",
                                                    pos_y=f"{center_y+y/2}",
                                                    chip="Q_chip",
                                               connection_pads=cp[i]))]
    qubits.append(a)  
qubits=qubits

q5=[TransmonPocket(design,'Q5', options=dict(pos_x=f"{center_x-2*x}mm",
                                                    pos_y=f"{center_y-y/2}",
                                                    chip="Q_chip",
                                               connection_pads=cp5))]
qubits.append(q5)                                              
for i,j in zip(range(6,9), space[1:4]):
    a=[TransmonPocket6(design,'Q'+str(i), options=dict(pos_x=f"{center_x+j}mm",
                                                    pos_y=f"{center_y-y/2}",
                                                    chip="Q_chip",
                                               connection_pads=cp678))]
    qubits.append(a)  

q9=[TransmonPocket(design,'Q9', options=dict(pos_x=f"{center_x+2*x}mm",
                                                    pos_y=f"{center_y-y/2}",
                                                    chip="Q_chip",
                                               connection_pads=cp9))]
qubits.append(q9)    
                                                                                          
design.rebuild()
gui.rebuild()
gui.autoscale()

# Launchpad

In [5]:
q5x=float(qubits[5][0].options.pos_x.strip('mm'))
q9x=float(qubits[9][0].options.pos_x.strip('mm'))
q0x=float(qubits[0][0].options.pos_x.strip('mm'))
q0y=float(qubits[0][0].options.pos_y.strip('mm'))
q1x=float(qubits[1][0].options.pos_x.strip('mm'))
q3x=float(qubits[3][0].options.pos_x.strip('mm'))
q3y=float(qubits[3][0].options.pos_y.strip('mm'))
q4x=float(qubits[4][0].options.pos_x.strip('mm'))
q2x=float(qubits[2][0].options.pos_x.strip('mm'))
q6x=float(qubits[6][0].options.pos_x.strip('mm'))
q7x=float(qubits[7][0].options.pos_x.strip('mm'))
q8x=float(qubits[8][0].options.pos_x.strip('mm'))


In [6]:
lp=[]

lp.append(LaunchpadWirebondCoupled(design, 'lp1',
                                         options = dict(chip="C_chip",
                                         pos_x = str(-4+0.25)+'mm',
                                         pos_y = str(4-0.05-0.04)+'mm',
                                         lead_length = '30um')))

lp.append(LaunchpadWirebondCoupled(design, 'lp2',
                                         options = dict(chip="C_chip",
                                         pos_x = str(4-0.25)+'mm',
                                         pos_y = str(4-0.05-0.04)+'mm',
                                         orientation = 180,
                                         lead_length = '30um')))

lp.append(LaunchpadWirebondCoupled(design, 'lp3',
                                         options = dict(chip="C_chip",
                                         pos_x = f'{q6x}mm',
                                         pos_y = '0mm',
                                         orientation = 0,
                                         lead_length = '30um')))

lp.append(LaunchpadWirebondCoupled(design, 'lp4',
                                         options = dict(chip="C_chip",
                                         pos_x = f'{q8x}mm',
                                         pos_y = '0mm',
                                         orientation = 180,
                                         lead_length = '30um')))
lp.append(LaunchpadWirebondCoupled(design, 'lp5',
                                         options = dict(chip="C_chip",
                                         pos_x = f'{q5x-0.3}mm',
                                         pos_y = str(-4+0.05+0.2)+'mm',
                                         orientation = 90,
                                         lead_length = '30um')))
lp.append(LaunchpadWirebondCoupled(design, 'lp6',
                                         options = dict(chip="C_chip",
                                         pos_x = f'{q9x+0.3}mm',
                                         pos_y = str(-4+0.05+0.2)+'mm',
                                         orientation = 90,
                                         lead_length = '30um')))
# gui.rebuild()
# gui.autoscale()                                         
                                     

# Readout resonator

In [7]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [8]:
resonator_length_2=[]
for i in range(4):
    a=find_resonator_length(7.0+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)
    a=a-0.118
    resonator_length_2.append(a)
print(resonator_length_2)


[8.576776247529553, 8.335254683870284, 8.10678833984142, 7.89034654020913]


In [17]:
PadWidth=float(c3.options.pad_width.strip('um'))*0.001
cpwStubHeight=float(c3.options.cpw_stub_height.strip('um'))*0.001
resonator_length_4=[]
for i in range(4):
    a=find_resonator_length(7.0+i*0.2,10,6,4)
    a=a.strip('mm')
    a=float(a)
    a=f"{a-0.118-PadWidth-cpwStubHeight-0.6+0.22}mm"
    resonator_length_4.append(a)
print(resonator_length_4)


['3.499388123764776mm', '3.378627341935142mm', '3.2643941699207106mm', '3.1561732701045653mm']


In [9]:
from collections import OrderedDict
jogs2=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            asymmetry='0 um', start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension
        ),
        total_length=length,
    fillet = '99.9um')
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [10]:
jogs8=OrderedDict()
jogs8[0]=['R','1.4mm']
jogs8[1]=['L','200um']

jogs6=OrderedDict()
jogs6[0]=['L','1.4mm']
jogs6[1]=['R','200um']

jogs5=OrderedDict()
jogs5[0]=['L','0.5mm']
jogs9=OrderedDict()
jogs9[0]=['R','0.5mm']

jogs0=OrderedDict()
jogs0[0]=["R",'0.5mm']
jogs0[1]=['R', '0.525mm']
jogs0[2]=['L', '0.5mm']
jogs0[3]=["L",'0.5mm']
jogs0[4]=['R', '0.5mm']
jogs0[5]=['R', '0.475mm']
jogs0[6]=['L', '0.4mm']

jogs04=OrderedDict()
jogs04[0]=["L",'0.5mm']
jogs04[1]=['L', '0.665mm']
jogs04[2]=['R', '0.5mm']
jogs04[3]=["R",'0.665mm']
jogs04[4]=['L', '0.5mm']
jogs04[5]=['L', '0.5mm']
jogs04[6]=['R', '0.4mm']


jogs01=OrderedDict()
jogs01[0]=["L",'0.4mm']
jogs01[1]=['L', '0.4mm']
jogs01[2]=['R', '0.4mm']
jogs01[3]=["R",'0.4mm']
jogs01[4]=['L', '0.4mm']
jogs01[5]=['L', '0.3mm']





# Connect multiplexed Feedline

In [11]:
# Coupled line tee
clt0 = CoupledLineTee(design, "clt0", options=dict(pos_x=f"{q0x-0.5}mm",pos_y=str(4-0.05-0.04)+'mm', chip="C_chip",down_length='26um', mirror=True,open_termination=False))
clt1 = CoupledLineTee(design, "clt1", options=dict(pos_x=f"{q1x+0.3}mm", pos_y=str(4-0.05-0.04)+'mm',chip="C_chip",down_length='26um', mirror=False,open_termination=False))
clt3 = CoupledLineTee(design, "clt3", options=dict(pos_x=f"{q3x-0.3}mm", pos_y=str(4-0.05-0.04)+'mm',chip="C_chip",down_length='26um', mirror=True,open_termination=False))
clt4 = CoupledLineTee(design, "clt4", options=dict(pos_x=f"{q4x+0.5}mm", pos_y=str(4-0.05-0.04)+'mm',chip="C_chip",down_length='26um',open_termination=False))
clt2 = CoupledLineTee(design, "clt2", options=dict(pos_x=f"{q2x}mm", pos_y='0mm',chip="C_chip", orientation='180',down_length='26um',open_termination=False))
clt6 = CoupledLineTee(design, "clt6", options=dict(pos_x=f"{q6x+0.3}mm", pos_y='0mm', chip="C_chip",down_length='26um', mirror=True,open_termination=False))
clt7 = CoupledLineTee(design, "clt7", options=dict(pos_x=f"{q7x+0.3}mm", pos_y='0mm',chip="C_chip",down_length='26um',open_termination=False))
clt8 = CoupledLineTee(design, "clt8", options=dict(pos_x=f"{q8x-0.3}mm", pos_y='0mm',chip="C_chip",down_length='26um',open_termination=False))


# design.rebuild()
# gui.rebuild()
# gui.autoscale()

In [13]:
mr1=RouteStraight(design, 'mr1',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lp1',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
mr2=RouteStraight(design, 'mr2',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='clt1',
                                                            pin='prime_start'))))
mr3=RouteStraight(design, 'mr3',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt1',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='clt3',
                                                            pin='prime_start'))))
mr4=RouteStraight(design, 'mr4',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt3',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='clt4',
                                                            pin='prime_start')))) 
mr5=RouteStraight(design, 'mr5',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt4',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lp2',
                                                            pin='tie'))))
mr6=RouteStraight(design, 'mr6',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='lp3',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt6',
                                                            pin='prime_start'))))
mr7=RouteStraight(design, 'mr7',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt6',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='clt2',
                                                            pin='prime_end'))))
mr8=RouteStraight(design, 'mr8',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt2',
                                                            pin='prime_start'),
                                                            end_pin=Dict(
                                                            component='clt7',
                                                            pin='prime_start'))))
mr9=RouteStraight(design, 'mr9',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt7',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='clt8',
                                                            pin='prime_start')))) 
mr10=RouteStraight(design, 'mr10',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt8',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lp4',
                                                            pin='tie'))))                                                            
          

                            

In [14]:
c3 = LeftCoupler(design, "c3", options=dict(pos_x=f"{q3x-0.155}mm",
                                              pos_y=f"{q3y+0.06}mm",
                                              pad_width="150um",
                                              pad_height="80um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))


In [15]:
jogs03=OrderedDict()
jogs03[0]=["R",'0.7mm']
jogs03[1]=['R', '0.7mm']
jogs03[2]=['L', '0.6mm']
jogs03[3]=["L",'0.42mm']
jogs03[4]=['R', '0.4mm']
# jogs03[5]=['R', '0.4mm']
# jogs03[6]=['L', '0.2mm']
# jogs03[7]=['L', '0.38mm']
#jogs03[8]=['R', '0.4mm']

In [ ]:

#rr0=connect('rr0', 'clt0', 'second_end','qc1', 'in',  resonator_length_4[0],jogs2,jogs2, '+um', '0.625mm', '0.1mm')
# rr1=connect('rr1', 'Q1', 'b', 'clt1', 'second_end', resonator_length_4[1], jogs01,jogs2, '00um', '0.2mm', '0.1mm')
rr3=connect('rr3','c3','in', 'clt3', 'second_end', resonator_length_4[2], jogs03,jogs2, '+um', '0.2mm', '0.1mm')
# rr4=connect('rr4', 'Q4', 'b', 'clt4', 'second_end', resonator_length_4[0], jogs04,jogs2, '-um', '0.626mm', '0.1mm',flip=True)
# rr2=connect('rr2', 'Q2', 'c', 'clt2', 'second_end', resonator_length_4[0], jogs2,jogs2, '00um', '100um', '0.mm')
# rr6=connect('rr6', 'Q6', 'a', 'clt6', 'second_end', resonator_length_4[2], jogs2,jogs2, '00um', '0.1mm', '0.1mm',flip=False)
# rr7=connect('rr7', 'Q7', 'a', 'clt7', 'second_end', resonator_length_4[3], jogs2, jogs2, '00um', '0.1mm', '0.1mm')
# rr8=connect('rr8', 'Q8', 'a', 'clt8', 'second_end', resonator_length_4[1], jogs2,jogs2, '00um', '0.1mm', '0.1mm',flip=False)
# rr5=connect('rr5', 'lp5', 'tie', 'Q5', 'b',  resonator_length_2[0], jogs5,jogs2, '00um', '0.1mm', '0.1mm')
# rr9=connect('rr9', 'lp6', 'tie', 'Q9', 'c', resonator_length_2[0], jogs9,jogs2, '00um', '0.1mm', '0.1mm')


# Bus Coupler

In [118]:
#10.97
jogs1=OrderedDict()
jogs1[0] = ["L", '200um']
jogs1[1] = ["L", '400um']
jogs1[2] = ["R", '200um']
jogs1[3] = ["R", '400um']
jogs1[4] = ["L", '200um']
jogs1[5] = ["L", '400um']
jogs1[6] = ["R", '200um']
jogs1[7] = ["R", '400um']
jogs1[8] = ["L", '200um']
jogs1[9] = ["L", '400um']
jogs1[10] = ["R", '200um']
jogs1[11] = ["R", '400um']
jogs1[12] = ["L", '200um']
jogs1[13] = ["L", '400um']
jogs1[14] = ["R", '200um']
jogs1[15] = ["R", '400um']
jogs1[16] = ["L", '200um']
jogs1[17] = ["L", '400um']
jogs1[18] = ["R", '200um']
jogs1[19] = ["R", '900um']
#
jogs1[20] = ["R", '200um']
jogs1[21] = ["R", '400um']
jogs1[22] = ["L", '200um']
jogs1[23] = ["L", '400um']
jogs1[24] = ["R", '200um']
jogs1[25] = ["R", '400um']
jogs1[26] = ["L", '200um']
jogs1[27] = ["L", '400um']
jogs1[28] = ["R", '200um']
jogs1[29] = ["R", '400um']
jogs1[30] = ["L", '200um']
jogs1[31] = ["L", '400um']
jogs1[32] = ["R", '200um']
jogs1[33] = ["R", '400um']
jogs1[34] = ["L", '200um']
jogs1[35] = ["L", '400um']
jogs1[36] = ["R", '200um']
jogs1[37] = ["R", '400um']

jogs2=OrderedDict()
jogs3=OrderedDict()
jogs3[0] = ["R", '300um']
jogs3[1] = ["R", '400um']
jogs3[2] = ["L", '200um']
jogs3[3] = ["L", '400um']
jogs3[4] = ["R", '200um']
jogs3[5] = ["R", '400um']
jogs3[6] = ["L", '200um']
jogs3[7] = ["L", '400um']
jogs3[8] = ["R", '200um']
jogs3[9] = ["R", '400um']
jogs3[10] = ["L", '200um']
jogs3[11] = ["L", '400um']
jogs3[12] = ["R", '200um']
jogs3[13] = ["R", '400um']
jogs3[14] = ["L", '200um']
jogs3[15] = ["L", '400um']
jogs3[16] = ["R", '200um']
jogs3[17] = ["R", '400um']
jogs3[18] = ["L", '200um']
jogs3[19] = ["L", '900um']

jogs3[20] = ["L", '200um']
jogs3[21] = ["L", '400um']
jogs3[22] = ["R", '200um']
jogs3[23] = ["R", '400um']
jogs3[24] = ["L", '200um']
jogs3[25] = ["L", '400um']
jogs3[26] = ["R", '200um']
jogs3[27] = ["R", '400um']
jogs3[28] = ["L", '200um']
jogs3[29] = ["L", '400um']
jogs3[30] = ["R", '200um']
jogs3[31] = ["R", '400um']
jogs3[32] = ["L", '200um']
jogs3[33] = ["L", '400um']
jogs3[34] = ["R", '200um']
jogs3[35] = ["R", '400um']
jogs3[36] = ["L", '200um']
jogs3[37] = ["L", '400um']

In [119]:
# 11.05
jogs1=OrderedDict()
jogs1[0] = ["L", '300um']
jogs1[1] = ["L", '500um']
jogs1[2] = ["R", '200um']
jogs1[3] = ["R", '500um']
jogs1[4] = ["L", '200um']
jogs1[5] = ["L", '500um']
jogs1[6] = ["R", '200um']
jogs1[7] = ["R", '500um']
jogs1[8] = ["L", '200um']
jogs1[9] = ["L", '500um']
jogs1[10] = ["R", '200um']
jogs1[11] = ["R", '500um']
jogs1[12] = ["L", '200um']
jogs1[13] = ["L", '500um']
jogs1[14] = ["R", '200um']
jogs1[15] = ["R", '1100um']

jogs1[16] = ["R", '200um']
jogs1[17] = ["R", '500um']
jogs1[18] = ["L", '200um']
jogs1[19] = ["L", '500um']
jogs1[20] = ["R", '200um']
jogs1[21] = ["R", '500um']
jogs1[22] = ["L", '200um']
jogs1[23] = ["L", '500um']
jogs1[24] = ["R", '200um']
jogs1[25] = ["R", '500um']
jogs1[26] = ["L", '200um']
jogs1[27] = ["L", '500um']
jogs1[28] = ["R", '200um']
jogs1[29] = ["R", '500um']
jogs1[30] = ["L", '300um']
jogs1[31] = ["L", '500um']

jogs2=OrderedDict()
jogs3=OrderedDict()
jogs3[0] = ["R", '300um']
jogs3[1] = ["R", '400um']
jogs3[2] = ["L", '200um']
jogs3[3] = ["L", '400um']
jogs3[4] = ["R", '200um']
jogs3[5] = ["R", '400um']
jogs3[6] = ["L", '200um']
jogs3[7] = ["L", '400um']
jogs3[8] = ["R", '200um']
jogs3[9] = ["R", '400um']
jogs3[10] = ["L", '200um']
jogs3[11] = ["L", '400um']
jogs3[12] = ["R", '200um']
jogs3[13] = ["R", '400um']
jogs3[14] = ["L", '200um']
jogs3[15] = ["L", '400um']
jogs3[16] = ["R", '200um']
jogs3[17] = ["R", '400um']
jogs3[18] = ["L", '200um']
jogs3[19] = ["L", '900um']

jogs3[20] = ["L", '200um']
jogs3[21] = ["L", '400um']
jogs3[22] = ["R", '200um']
jogs3[23] = ["R", '400um']
jogs3[24] = ["L", '200um']
jogs3[25] = ["L", '400um']
jogs3[26] = ["R", '200um']
jogs3[27] = ["R", '400um']
jogs3[28] = ["L", '200um']
jogs3[29] = ["L", '400um']
jogs3[30] = ["R", '200um']
jogs3[31] = ["R", '400um']
jogs3[32] = ["L", '200um']
jogs3[33] = ["L", '400um']
jogs3[34] = ["R", '200um']
jogs3[35] = ["R", '400um']
jogs3[36] = ["L", '200um']
jogs3[37] = ["L", '400um']

In [41]:
bc01=connect('bc01','Q0','b','Q1','a','9mm',jogs1,jogs2,'0mm','0.1mm','0.1mm' )
bc34=connect('bc34','Q3','b','Q4','a', '9mm',jogs1,jogs2,'0um','0.1mm','0.1mm')
bc56=connect('bc56','Q5','c','Q6','b', '9mm',jogs3,jogs2,'0um','0.1mm','0.1mm')
bc67=connect('bc67','Q6','c','Q7','b','9mm',jogs3,jogs2,'0mm','0.1mm','0.1mm' )
bc78=connect('bc78','Q7','c','Q8','b', '9mm',jogs3,jogs2,'0um','0.1mm','0.1mm')
bc89=connect('bc89','Q8','c','Q9','b','9mm',jogs3,jogs2,'0mm','0.1mm','0.1mm')

In [42]:
bc50=connect('bc50','Q5','a','Q0','c','8.64mm',jogs2,jogs2,'0mm','0.5mm','100um' )
bc94=connect('bc94','Q9','a','Q4','c','8.64mm',jogs2, jogs2, '0um','0.5mm','100um' )


In [120]:
jogs12=OrderedDict()
jogs12[0]=['R','1.1mm']
jogs12[1]=['L','1.2mm']

jogs12[2]=['L','0.2mm']
jogs12[3]=['L','1.mm']
jogs12[4]=['R','0.2mm']
jogs12[5]=['R','1.mm']
jogs12[6]=['L','0.2mm']
jogs12[7]=['L','1.mm']
jogs12[8]=['R','0.2mm']
jogs12[9]=['R','0.7mm']
jogs12[10]=['L','0.2mm']
jogs12[11]=['L','0.7mm']

jogs12[12]=['R','0.8mm']
jogs12[13]=['L','0.5mm']


jogs13=OrderedDict()
jogs13[0]=['L','1.1mm']
jogs13[1]=['R','1.2mm']

jogs13[2]=['R','0.2mm']
jogs13[3]=['R','1.mm']
jogs13[4]=['L','0.2mm']
jogs13[5]=['L','1.mm']
jogs13[6]=['R','0.2mm']
jogs13[7]=['R','1.mm']
jogs13[8]=['L','0.2mm']
jogs13[9]=['L','0.7mm']
jogs13[10]=['R','0.2mm']
jogs13[11]=['R','0.7mm']

jogs13[12]=['L','0.8mm']
jogs13[13]=['R','0.5mm']



In [43]:
bc12=connect('bc12','Q1','c','Q2','a','8.84mm',jogs12,jogs2,'0mm','100um','100um' )
bc32=connect('bc32','Q3','c','Q2','b','8.84mm',jogs13, jogs2, '0um','100um','100um', flip=True )

01:10PM 58s INFO [connect_meandered]: Zero meanders for rr0
01:10PM 58s INFO [connect_meandered]: Zero meanders for rr4
01:10PM 58s WARNING [check_lengths]: For path table, component=rr4, key=trace has short segments that could cause issues with fillet. Values in (8-9)  are index(es) in shapely geometry.
01:10PM 59s WARNING [check_lengths]: For path table, component=rr4, key=cut has short segments that could cause issues with fillet. Values in (8-9)  are index(es) in shapely geometry.


# LOM Analysis

In [19]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [20]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.5',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.5',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [21]:
c1.sim.run(components=['Q3','c3'], open_terminations=[('Q3', 'b'), ('Q3', 'c'), ('c3', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 08:59AM [connect_project]: Connecting to Ansys Desktop API...
INFO 08:59AM [load_ansys_project]: 	Opened Ansys App
INFO 08:59AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 08:59AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project17
INFO 08:59AM [connect_design]: No active design found (or error getting active design).
INFO 08:59AM [connect]: 	 Connected to project "Project17". No design detected
INFO 08:59AM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 08:59AM [connect_setup]: 	No design setup detected.
WARNING 08:59AM [connect_setup]: 	Creating Q3D default setup.
INFO 08:59AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 08:59AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 08:59AM [analyze]: Analyzing setup Setup
INFO 09:01AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\T

,b_connector_pad_Q3,c_connector_pad_Q3,ground_C_chip_plane,ground_Q_chip_plane,pad_bot_Q3,pad_top_Q3,quad_coupler_c3
b_connector_pad_Q3,46.03700,-0.14543,-8.2734,-25.3730,-1.2482,-10.7970,0.0000
c_connector_pad_Q3,-0.14543,51.33600,-10.7610,-26.2670,-12.5010,-1.4492,0.0000
ground_C_chip_plane,-8.27340,-10.76100,782.9800,-581.8000,-34.8490,-28.8060,-75.6000
ground_Q_chip_plane,-25.37300,-26.26700,-581.8000,728.0900,-33.1310,-32.5370,-1.0447
pad_bot_Q3,-1.24820,-12.50100,-34.8490,-33.1310,111.3800,-29.4740,0.0000
pad_top_Q3,-10.79700,-1.44920,-28.8060,-32.5370,-29.4740,110.2200,-6.3940
quad_coupler_c3,0.00000,0.00000,-75.6000,-1.0447,0.0000,-6.3940,83.2180


In [31]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.4
c1.setup.freq_bus = [7.2,7.2]
c1.run_lom()
c1.lumped_oscillator_all

[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 6.040250 [GHz]
EC 345.357924 [MHz]
EJ 14.854145 [GHz]
alpha -402.279990 [MHz]
dispersion 47.294676 [KHz]
Lq 10.995564 [nH]
Cq 56.087400 [fF]
T1 88.069055 [us]

**Coupling Properties**

tCqbus1 -3.059693 [fF]
gbus1_in_MHz -56.001218 [MHz]
χ_bus1 -1.067527 [MHz]
1/T1bus1 1014.059721 [Hz]
T1bus1 156.948294 [us]

tCqbus2 -5.001602 [fF]
gbus2_in_MHz -88.910493 [MHz]
χ_bus2 -3.548254 [MHz]
1/T1bus2 352.279261 [Hz]
T1bus2 451.786298 [us]

tCqbus3 5.597628 [fF]
gbus3_in_MHz 99.458296 [MHz]
χ_bus3 -4.440079 [MHz]
1/T1bus3 440.821803 [Hz]
T1bus3 361.041450 [us]
Bus-Bus Couplings
gbus1_2 2.619322 [MHz]
gbus1_3 3.002673 [MHz]
gbus2_3 6.304771 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.365092,268.346533,14.854145,-305.578145,4.260594,"[-54.26617065392253, -74.47369326035218, 83.68...","[-0.38920098630004063, -0.8850458997439056, -1...",0.389201,54.266171
2,6.508241,405.369499,14.854145,-480.357069,186.222136,"[-75.4925853657845, -105.41311038905727, 74.51...","[-4.504025690560571, -13.225010072997646, -6.6...",4.504026,75.492585
3,6.254713,372.167702,14.854145,-436.858245,90.959853,"[-70.43512210806097, -86.31931496695807, 95.61...","[-2.4161646515595705, -5.0199149062728115, -6....",2.416165,70.435122
4,6.204873,365.833938,14.854145,-428.645679,78.45595,"[-68.06285449867315, -86.71277328002874, 97.97...","[-2.0686440138064217, -4.586681783263616, -5.8...",2.068644,68.062854
5,6.161437,360.365213,14.854145,-421.576497,68.831992,"[-65.1307724357619, -86.92038477225083, 96.754...","[-1.7595392560006573, -4.2375539841427186, -5....",1.759539,65.130772
6,6.134679,357.019845,14.854145,-417.261976,63.440739,"[-63.86893043753286, -86.52487044125273, 98.82...","[-1.6181454398209623, -3.9919488438943067, -5....",1.618145,63.868930
7,6.111805,354.174466,14.854145,-413.598182,59.134188,"[-62.129918144521746, -86.73188241915649, 98.2...","[-1.4745797481116214, -3.8438228318463845, -4....",1.474580,62.129918
8,6.069288,348.920409,14.854145,-406.84709,51.816527,"[-59.17156234130842, -87.28274056942341, 97.78...","[-1.2483431079361482, -3.601933634475453, -4.5...",1.248343,59.171562
9,6.065239,348.422322,14.854145,-406.208036,51.163597,"[-57.64303614800873, -87.81822134704264, 98.68...","[-1.177006890178573, -3.6197494988074426, -4.5...",1.177007,57.643036
10,6.04025,345.357924,14.854145,-402.27999,47.294676,"[-56.00121800534322, -88.91049289940693, 99.45...","[-1.0675265216937184, -3.5482537260279163, -4....",1.067527,56.001218


In [24]:
c1.sim.run(components=['Q3','c3'], open_terminations=[('Q3', 'b'), ('Q3', 'c')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 06:26PM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:26PM [load_ansys_project]: 	Opened Ansys App
INFO 06:26PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 06:26PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project16
INFO 06:26PM [connect_design]: No active design found (or error getting active design).
INFO 06:26PM [connect]: 	 Connected to project "Project16". No design detected
INFO 06:26PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 06:26PM [connect_setup]: 	No design setup detected.
WARNING 06:26PM [connect_setup]: 	Creating Q3D default setup.
INFO 06:26PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 06:26PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 06:26PM [analyze]: Analyzing setup Setup
INFO 06:27PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\T

,b_connector_pad_Q3,c_connector_pad_Q3,ground_C_chip_plane,ground_Q_chip_plane,pad_bot_Q3,pad_top_Q3
b_connector_pad_Q3,45.96200,-0.14568,-8.2743,-25.222,-1.246,-10.8710
c_connector_pad_Q3,-0.14568,51.22200,-10.7420,-26.149,-12.502,-1.4485
ground_C_chip_plane,-8.27430,-10.74200,716.3500,-583.290,-34.783,-35.2400
ground_Q_chip_plane,-25.22200,-26.14900,-583.2900,727.820,-33.126,-32.5310
pad_bot_Q3,-1.24600,-12.50200,-34.7830,-33.126,111.390,-29.4660
pad_top_Q3,-10.87100,-1.44850,-35.2400,-32.531,-29.466,110.3100


In [29]:
c1.setup

{'junctions': {'Lj': 11, 'Cj': 2}, 'freq_readout': 7.4, 'freq_bus': [7.2, 7.2]}

In [25]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.4
c1.setup.freq_bus = [7.2,7.2]
c1.run_lom()
c1.lumped_oscillator_all

[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 4.491259 [GHz]
EC 184.549975 [MHz]
EJ 14.854145 [GHz]
alpha -204.683065 [MHz]
dispersion 0.063322 [KHz]
Lq 10.995564 [nH]
Cq 104.959256 [fF]
T1 49.163302 [us]

**Coupling Properties**

tCqbus1 -22.976935 [fF]
gbus1_in_MHz -252.710886 [MHz]
χ_bus1 -3.074923 [MHz]
1/T1bus1 3187.205438 [Hz]
T1bus1 49.935577 [us]

tCqbus2 1.524337 [fF]
gbus2_in_MHz 16.501583 [MHz]
χ_bus2 -0.014955 [MHz]
1/T1bus2 1.577329 [Hz]
T1bus2 100901.525163 [us]

tCqbus3 -8.456500 [fF]
gbus3_in_MHz -91.492121 [MHz]
χ_bus3 -0.459735 [MHz]
1/T1bus3 48.488482 [Hz]
T1bus3 3282.324731 [us]
Bus-Bus Couplings
gbus1_2 38.214330 [MHz]
gbus1_3 13.602325 [MHz]
gbus2_3 4.512409 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,4.57511,191.843044,14.854145,-213.285052,0.101783,"[-210.33034560073727, 44.2046795536382, -80.32...","[-2.3327511057709254, -0.11800883815738535, -0...",2.332751,210.330346
2,4.643756,197.92968,14.854145,-220.490455,0.148185,"[-167.29295109084015, 27.01659621107877, -68.0...","[-1.5909121099135322, -0.04768467032494244, -0...",1.590912,167.292951
3,4.57354,191.705026,14.854145,-213.121944,0.100898,"[-235.7502898951985, 14.2486978096393, -90.187...","[-2.925674184830679, -0.012239157400730215, -0...",2.925674,235.750290
4,4.559468,190.470698,14.854145,-211.66377,0.093283,"[-242.08603354384977, 14.915658376013768, -91....","[-3.038148599645026, -0.013198739742030445, -0...",3.038149,242.086034
5,4.538827,188.668205,14.854145,-209.53617,0.083068,"[-242.78051755081805, 14.912122210402124, -90....","[-2.987867049698102, -0.012887037863726886, -0...",2.987867,242.780518
6,4.533396,188.195491,14.854145,-208.978543,0.080556,"[-243.39350875245057, 15.350980978823166, -90....","[-2.985340263574076, -0.01357295979678436, -0....",2.985340,243.393509
7,4.545037,189.209533,14.854145,-210.174914,0.086027,"[-243.58033983059414, 15.525009417710981, -91....","[-3.0279227061124385, -0.014066811691772482, -...",3.027923,243.580340
8,4.504247,185.669488,14.854145,-206.001259,0.068233,"[-246.13157722309273, 15.876494934094648, -92....","[-2.9580847320759838, -0.014047714038058984, -...",2.958085,246.131577
9,4.500039,185.30634,14.854145,-205.573574,0.066604,"[-246.60767991559194, 16.133583188758674, -91....","[-2.956072018219846, -0.014437678663454611, -0...",2.956072,246.607680
10,4.491259,184.549975,14.854145,-204.683065,0.063322,"[-252.71088555872393, 16.501582847555706, -91....","[-3.074922875385506, -0.014955177073930119, -0...",3.074923,252.710886


# EPR Analysis

In [2]:
from qiskit_metal.analyses.quantization import EPRanalysis

In [1]:
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

NameError: name 'EPRanalysis' is not defined

In [186]:
hfss.start()

INFO 03:42PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:42PM [load_ansys_project]: 	Opened Ansys App
INFO 03:42PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 03:42PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project16
INFO 03:42PM [connect_design]: No active design found (or error getting active design).
INFO 03:42PM [connect]: 	 Connected to project "Project16". No design detected


True

In [272]:
hfss.activate_ansys_design("rr3-3", 'eigenmode')
hfss.render_design(['rr3','c3','lp1','lp2','mr1','mr2','mr3','mr4','mr5','clt3','clt0','clt1','clt4'], 
                    [('rr3','start')], 
                    box_plus_buffer=False)

05:20PM 40s WARNING [activate_ansys_design]: The design_name=rr3-3 was not in active project.  Designs in active project are: 
['rr3', 'Design_q3d', 'rr3-2', 'rr3-1'].  A new design will be added to the project.  
INFO 05:20PM [connect_design]: 	Opened active design
	Design:    rr3-3 [Solution type: Eigenmode]
WARNING 05:20PM [connect_setup]: 	No design setup detected.
WARNING 05:20PM [connect_setup]: 	Creating eigenmode default setup.
INFO 05:20PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [73]:
#from turtle import end_fill


#hfss.activate_ansys_design("TransmonReadout", 'eigenmode')
# hfss.render_design(['rr0','rr1','rr3','rr4','rr2','rr6','rr7','rr8','rr5','rr9'], 
#                     [('rr0','start'),('rr1','start'),('rr3','start'),('rr4','start'),
#                      ('rr2','start'),('rr6','start'),('rr7','start'),('rr8','start'),
#                      ('rr5','start'),('rr9','start'),('rr5','end'),('rr9','end')], 
#                     box_plus_buffer=False)
# hfss.activate_ansys_design("Test2", 'eigenmode')
# hfss.render_design(['rr4','rr8'], 
#                     [('rr4','start'),('rr8','start')], 
#                     box_plus_buffer=False)


10:16AM 42s WARNING [activate_ansys_design]: The design_name=Test0 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 10:16AM [connect_design]: 	Opened active design
	Design:    Test0 [Solution type: Eigenmode]
WARNING 10:16AM [connect_setup]: 	No design setup detected.
WARNING 10:16AM [connect_setup]: 	Creating eigenmode default setup.
INFO 10:16AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [287]:
hfss.close()

Warning! 6 COM references still alive
Ansys will likely refuse to shut down


True